<a href="https://colab.research.google.com/github/s528661/nlp100/blob/main/nlp100_9_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 4.8 MB/s eta 0:00:00


In [2]:
import re,sys,os

import pickle
import time
import copy

import math
import random
import string

from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import scipy as scp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn import feature_extraction, preprocessing

from sklearn import svm
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, f1_score
from sklearn.metrics import accuracy_score,  precision_score, recall_score, confusion_matrix, roc_auc_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import RidgeClassifier

In [4]:
import MeCab

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [5]:
import tensorflow as tf
from tensorflow import keras, nn
from keras import layers, optimizers, losses

In [6]:
if not os.path.isfile('./NewsAggregatorDataset.zip'):
  !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip --no-check-certificate
  !unzip ./NewsAggregatorDataset.zip

--2024-01-06 03:17:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas     [         <=>        ]  27.87M  11.4MB/s    in 2.5s    

2024-01-06 03:17:24 (11.4 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203]

Archive:  ./NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


# 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [7]:
data0 = pd.read_csv('newsCorpora.csv',delimiter='\t',header=None,
                 names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])

In [8]:
data = data0[data0['PUBLISHER'].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]

In [9]:
data

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
12,13,Europe reaches crunch point on banking union,http://in.reuters.com/article/2014/03/10/eu-ba...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501755
13,14,ECB FOCUS-Stronger euro drowns out ECB's messa...,http://in.reuters.com/article/2014/03/10/ecb-p...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470501948
19,20,"Euro Anxieties Wane as Bunds Top Treasuries, S...",http://www.businessweek.com/news/2014-03-10/ge...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503148
20,21,Noyer Says Strong Euro Creates Unwarranted Eco...,http://www.businessweek.com/news/2014-03-10/no...,Businessweek,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,www.businessweek.com,1394470503366
29,30,REFILE-Bad loan triggers key feature in ECB ba...,http://in.reuters.com/article/2014/03/10/euroz...,Reuters,b,dPhGU51DcrolUIMxbRm0InaHGA2XM,in.reuters.com,1394470505070
...,...,...,...,...,...,...,...,...
422300,422819,UN: Ebola Could Eventually Infect 20000 People,http://www.huffingtonpost.com/2014/08/28/ebola...,Huffington Post,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.huffingtonpost.com,1409228423333
422301,422820,"Ebola toll tops 1550, continues to accelerate ...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228423827
422302,422821,"UPDATE 1-Ebola toll tops 1550, outbreak accele...",http://in.reuters.com/article/2014/08/28/healt...,Reuters,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,in.reuters.com,1409228424525
422309,422828,"Ebola Cases May Surpass 20000, WHO Says in Upd...",http://www.businessweek.com/news/2014-08-28/eb...,Businessweek,m,dhhnSHVoyA7ENBM3boDX_D_-3PV6M,www.businessweek.com,1409228426943


In [10]:
def preprocessing(text):
  text = "".join([i for i in text if i not in string.punctuation])
  text = text.lower()
  text = re.sub("[0-9]+", "", text)
  return text

In [11]:
data['pTITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))

<ipython-input-11-26ac3ffb61ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pTITLE'] = data['TITLE'].apply(lambda x: preprocessing(x))


In [12]:
train_val, test = train_test_split(data, test_size=0.2, shuffle=True)
train, val = train_test_split(train_val, test_size=0.1, shuffle=True)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

In [13]:
freq_words = defaultdict(int)
for title in train['pTITLE']:
  for word in title.split():
    freq_words[word] += 1

In [14]:
freq_words = sorted(freq_words.items(), key=lambda x:x[1], reverse=True)

In [15]:
db_words = []
for id, (word, count) in enumerate(freq_words):
  if count < 2:
    db_words.append([0,word])
  else:
    db_words.append([id+1,word])

# 81. RNNによる予測

In [52]:
class RNN(keras.Model):
  def __init__(self, dw, dh, n_words, n_class, n_unit):
    super().__init__()
    self.rnn = layers.SimpleRNN(n_unit, activation="tanh", input_shape=(dw, dh), return_sequences=True)
    # self.rnn = layers.RNN(cell=n_cell, activation="tanh", input_shape=(dw, dh), return_sequences=True)
    self.emb = layers.Embedding(n_words, dh)
    self.linear = layers.Dense(n_class, activation='relu')

  def __call__(self, x, training=True):
    x = self.emb(x)
    x = self.rnn(x)
    x = x[:, -1, :]
    #x = nn.softmax(x)
    x = self.linear(x)
    return x

# 82. 確率的勾配降下法による学習  
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [53]:
labels = list(set(data['CATEGORY']))
dsc_labels =  {k: i for i, k in enumerate(labels)}

In [54]:
dw = 300
dh = 50
n_words = len(db_words) + 1
n_class = len(labels)

In [55]:
def one_hot_encoding(series):
    le = LabelEncoder()
    lvl = le.fit_transform(series)

    ohe = OneHotEncoder()
    enced = ohe.fit_transform(lvl.reshape(-1,1)).astype(int)
    names = [(series.name+"_")+str(s) for s in le.classes_]
    r_df = pd.DataFrame(index=series.index, columns=names, data=enced.toarray(), dtype=np.int64)
    return r_df

In [56]:
def data2vec(data, X_name, y_name, n_class):
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(data[X_name]).toarray()
  #y = one_hot_encoding(data[y_name]).values
  y = data[y_name].map(lambda x: dsc_labels[x])
  y = tf.one_hot(y, n_class)
  #Xy = pd.concat([X,y], axis=1)
  #Xy = pd.DataFrame(Xy, columns=vectorizer.get_feature_names_out())
  return X, y

In [57]:
X_train, y_train = data2vec(train,'pTITLE','CATEGORY',n_class)
X_val, y_val = data2vec(val,'pTITLE','CATEGORY',n_class)
X_test, y_test = data2vec(test,'pTITLE','CATEGORY',n_class)

In [58]:
#model = keras.Sequential([
#    layers.Embedding(n_words, dh),
#    layers.SimpleRNN(n_unit, activation="tanh", input_shape=(dw, dh), return_sequences=True),
#    layers.Dense(n_label, activation="softmax")
#    nn.softmax(),
#])

In [59]:
n_unit = 1

In [60]:
model = RNN(dw, dh, n_words, n_class, n_unit)

In [61]:
#model.summary()

In [62]:
optim = optimizers.SGD(learning_rate=0.0001)

In [63]:
loss = losses.CategoricalCrossentropy()

In [64]:
model.compile(
    #loss="sparse_categorical_crossentropy",
    loss=loss,
    optimizer=optim,
    metrics=["accuracy"],
    run_eagerly=True,
)

In [65]:
batch_size = 64
epochs = 1

In [66]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

151/151 [==============================] - 4443s 29s/step - loss: 5.3420 - accuracy: 0.0983 - val_loss: 3.3270 - val_accuracy: 0.0655


In [71]:
y_pred = model.predict(X_test)

84/84 [==============================] - 845s 10s/step


In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [73]:
y_pred

array([[0.        , 0.10136534, 0.4431287 , 0.5047814 ],
       [0.        , 0.10136534, 0.4431287 , 0.5047814 ],
       [0.        , 0.10136534, 0.4431287 , 0.5047814 ],
       ...,
       [0.        , 0.10136534, 0.4431287 , 0.5047814 ],
       [0.        , 0.10136534, 0.4431287 , 0.5047814 ],
       [0.        , 0.10136534, 0.4431287 , 0.5047814 ]], dtype=float32)

In [74]:
y_test

<tf.Tensor: shape=(2668, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

In [72]:
"Accuracy: {}".format(accuracy_score(y_test, y_pred))

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-72-e246691e1791>, line 1)